In [1]:
import warnings
%run ConnectOracleDB.ipynb

table_name = 'DICT_DEVICE_MOBILE'
df = fetch_table_as_dataframe(table_name)

C:\Users\a.tsereteli\AppData\Local\Temp\ipykernel_10292\3979122280.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql_query,connection)  # without parameters [non-prepared statement]
C:\Users\a.tsereteli\AppData\Local\Temp\ipykernel_10292\3979122280.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql_query,connection)  # without parameters [non-prepared statement]


In [23]:
# Data analysis and wrangling
import pandas as pd
import numpy as np

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df.head(5)

,TAC_CODE,DEVICE_BRAND,DEVICE_NAME,DEVICE_TECH_NAME,NETW,LTE_CAT,DEVICE_TYPE,IS_SMART,IS_M2M,SIM_TYPE,SIM_DUAL,SIMS,IMEIS
0,01199900,Multi-Tech,MTSMC-G2-V,MTSMC-G2-V,None,No,Basic Phone,N,No,Mini SIM,No,1,1
1,01201100,LG,GR500,GR500,3G,No,Feature Phone,N,No,Mini SIM,No,1,1
2,01202000,Apple,iPhone 3G,A1241,3G,No,Smartphone,Y,No,Mini SIM,No,1,1
3,01202100,Apple,iPhone 3G,A1241,3G,No,Smartphone,Y,No,Mini SIM,No,1,1
4,01177300,Apple,iPhone 3G,A1241,3G,No,Smartphone,Y,No,Mini SIM,No,1,1


In [17]:
dev_name = df[['DEVICE_BRAND','DEVICE_NAME']].dropna()
dev_name["DEVICE_TITLE"]=dev_name['DEVICE_BRAND']+' '+dev_name['DEVICE_NAME']
arr1 = dev_name["DEVICE_TITLE"].unique()
arr1

array(['Multi-Tech MTSMC-G2-V', 'LG GR500', 'Apple iPhone 3G', ...,
       'Vivo V23 PRO', 'ZTE Grand X IN',
       'Hui Zhou Cong Communication Co Ltd Beeline PRO 4'], dtype=object)

In [18]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from Levenshtein import distance

In [19]:
# Sample list of device models
device_models = arr1

In [20]:
# Calculate pairwise Levenshtein distances
arr_len = len(device_models)
distance_matrix = np.zeros((arr_len, arr_len))

In [24]:
for i in tqdm(range(arr_len)):
    for j in range(arr_len):
        distance_matrix[i, j] = distance(device_models[i], device_models[j])
distance_matrix

  0%|          | 0/30587 [00:00<?, ?it/s]

array([[ 0., 20., 19., ..., 19., 19., 44.],
       [20.,  0., 14., ..., 11., 12., 45.],
       [19., 14.,  0., ..., 14., 14., 42.],
       ...,
       [19., 11., 14., ...,  0., 13., 41.],
       [19., 12., 14., ..., 13.,  0., 41.],
       [44., 45., 42., ..., 41., 41.,  0.]])

In [25]:
# Perform hierarchical clustering
linkage_matrix = linkage(distance_matrix, method='single')
# Set a threshold to cut the dendrogram into clusters
threshold = 18  # Adjust this threshold based on your data and desired level of similarity
cluster_assignments = fcluster(linkage_matrix, threshold, criterion='distance')

# Count the size of each cluster
cluster_sizes = np.bincount(cluster_assignments)

# Create a dictionary to store device models in each cluster
clusters = {}
for i, cluster_id in enumerate(cluster_assignments):
    device_model = device_models[i]
    
    if cluster_sizes[cluster_id] <= 2:  # If cluster size is small (adjust the threshold)
        # Assign the device model to another cluster with a larger size
        new_cluster_id = np.argmax(cluster_sizes)  # Find the cluster with the largest size
        cluster_sizes[new_cluster_id] += 1
        cluster_id = new_cluster_id
    if cluster_id not in clusters:
        clusters[cluster_id] = []
    
    
    clusters[cluster_id].append(device_model)

# Print the clusters

for cluster_id, device_models in clusters.items():
    print(f"Cluster {cluster_id}: {device_models}")

MemoryError: Unable to allocate 6.97 GiB for an array with shape (30587, 30587) and data type float64

TypeError: object of type 'builtin_function_or_method' has no len()